In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set(rc={'figure.figsize':(10,10)})

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
raw = pd.read_csv("/kaggle/input/chilled-eda/chilled_water_cleaned.csv", index_col = "timestamp", parse_dates = True)
raw.head(3)

In [ ]:
raw = raw.resample("W").mean()

In [ ]:
raw.head(3)

In [ ]:
col = raw.columns
print(col[400:])

In [ ]:
#creating dataframe for a single site

peacock = pd.DataFrame()
P = [col for col in raw.columns if 'Peacock' in col]
peacock[P] = raw[P]

moose = pd.DataFrame()
M = [col for col in raw.columns if 'Moose' in col]
moose[M] = raw[M]
 
bull = pd.DataFrame()
B = [col for col in raw.columns if 'Bull' in col]
bull[B] = raw[B]

hog = pd.DataFrame()
H = [col for col in raw.columns if 'Hog' in col]
hog[H] = raw[H]

eagle = pd.DataFrame()
E = [col for col in raw.columns if 'Eagle' in col]
eagle[E] = raw[E]

cockatoo = pd.DataFrame()
C = [col for col in raw.columns if 'Cockatoo' in col]
cockatoo[C] = raw[C]

panther = pd.DataFrame()
pan = [col for col in raw.columns if 'Panther' in col]
panther[pan] = raw[pan]

fox = pd.DataFrame()
f = [col for col in raw.columns if 'Fox' in col]
fox[f] = raw[f]

bobcat = pd.DataFrame()
bob = [col for col in raw.columns if 'Bobcat' in col]
bobcat[bob] = raw[bob]

crow = pd.DataFrame()
cr = [col for col in raw.columns if 'Crow' in col]
crow[cr] = raw[cr]

sites = [peacock, moose, bull, hog, eagle, cockatoo, panther, fox, bobcat, crow]

testing some of the site df's

In [ ]:
panther.head(3)

In [ ]:
crow.head(2)

In [ ]:
eagle.head(2)

In [ ]:
bobcat.head(2)

In [ ]:
panther.plot()

In [ ]:
eagle.plot()

In [ ]:
#summing the total chilled water consumption per week
for site in sites:
    site["Chilled_sum"] = site.sum(axis = 1)

In [ ]:
#checking sum correct
eagle.head(3)

In [ ]:
bull.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import SCORERS

import datetime as dt

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
#splitting df for a site into train and test sets
model_panther = panther.copy()

train = model_panther.iloc[0:(len(model_panther)-30)]
test = model_panther.iloc[len(train):(len(model_panther)-1)]

In [ ]:
test.plot()

In [ ]:
#checking if stationary or not 
from statsmodels.tsa.stattools import adfuller

result = adfuller(panther["Chilled_sum"])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

P-value bigger than .05 threshold, so data is non-stationary. It has some time dependent structure.

so do we differentiate so p< .05?

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from matplotlib import pyplot as plt

In [ ]:
endog = train["Chilled_sum"]

mod = sm.tsa.statespace.SARIMAX(endog=endog, simple_differencing = False)
model_fit = mod.fit()
model_fit.summary()

In [ ]:
train['Chilled_sum'].plot(figsize=(25,10))
model_fit.fittedvalues.plot()
plt.show()

In [ ]:
predict = model_fit.predict(start = len(train),end = len(train)+len(test)-1)
test['predicted'] = predict.values
test.tail(5)


In [ ]:
test['predicted'].plot(color = 'red')
test["Chilled_sum"].plot()

In [ ]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error

 
# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error
 
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)
 
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores
 
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    models = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n','c','t','ct']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [ ]:
data = panther.values

# data split
n_test = 4
# model configs
cfg_list = sarima_configs(seasonal=[0, 4])
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)